In [1]:
import sys
sys.path.append("..")
import os
from copy import deepcopy
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import datetime

In [2]:
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
from constant import *
from Params import *
from abs_util.util_general import *
from abs_util.util_waterfall import run_Accounts,BasicInfo_calculator,CR_calculator,NPV_calculator
from Deal import Deal
from RevolvingDeal import RevolvingDeal
from Statistics import Statistics
from ReverseSelection import ReverseSelection
from AssetsCashFlow import AssetsCashFlow
from APCF_adjuster import APCF_adjuster

logger = get_logger(__name__)

In [4]:
if os.path.isfile(wb_name):
  os.remove(wb_name)

In [5]:
#Initializing Project
RD = RevolvingDeal(ProjectName,dt_param['dt_pool_cut'],dt_param['dt_trust_effective'],Flag_RevolvingDeal,date_revolving_pools_cut)

INFO:Deal:Initializing Project....
INFO:Deal:ProjectName is: ABS13
INFO:Deal:date_pool_cut is: 2019-03-01
INFO:Deal:date_trust_effective is: 2019-05-20
INFO:RevolvingDeal:If it is RevolvingDeal: True
INFO:RevolvingDeal:date_revolving_pools_cut: [datetime.date(2019, 6, 1), datetime.date(2019, 7, 1), datetime.date(2019, 8, 1), datetime.date(2019, 9, 1), datetime.date(2019, 10, 1), datetime.date(2019, 11, 1)]


In [6]:
#Fetch asset pool
RD.get_AssetPool(['ABS13_InitialPool_20190311'])

INFO:Deal:Getting part ABS13_InitialPool_20190311...
INFO:Deal:Renaming header....
INFO:Deal:Asset Pool Gotten.


In [7]:
#RD.run_Stat(Distribution_By_Category,Distribution_By_Bins)

In [8]:
#pd.pivot_table(RD.asset_pool,values='Amount_Outstanding_yuan', index=['Gender'],columns=['Marriagestate'], aggfunc=np.sum)

#pivot_ui(RD.asset_pool[['Gender','Marriagestate','Province','Amount_Outstanding_yuan']],rows=['Gender'], cols=['Marriagestate'])

In [9]:
RD.init_oAP_Acc()

INFO:Deal:init_oAP_Acc....
INFO:Deal:init_oAP_Acc Done.
INFO:Deal:oAP_ACC accounts consist of:  
            Principal Collection Accounts:
            self.AP_PAcc_original_O,self.AP_PAcc_actual_O,self.AP_PAcc_pay_O,self.AP_PAcc_buy_O. 
            self.AP_PAcc_actual_O = [['ER_recycle_principal','Normal_recycle_principal','Redemption_recycle_principal',
                                      'Overdue_1_30_recycle_principal','Overdue_31_60_recycle_principal','Overdue_61_90_recycle_principal',
                                      'Recovery_recycle_principal']].sum(axis=1)
            
            The same type of accounts are also created for Interest Collection.


In [10]:
for asset_status in all_asset_status:
    if len(RD.asset_pool[(RD.asset_pool['贷款状态'] == asset_status)]) == 0:
        logger.info('No Assets to calc for {0}'.format(asset_status))
        continue
    else:
        logger.info('Collecting CF for asset_status {0}'.format(asset_status))   
        
        ####################
        RD.get_oAPCF(asset_status,
                     asset_status_calcDate_BackMonth[asset_status]['BackMonth'],
                     asset_status_calcDate_BackMonth[asset_status]['calcDate']
                     )

        save_to_excel(RD.apcf_original[asset_status],'cf_o',wb_name)
        #save_to_excel(RD.df_ppmt[asset_status],'df_ppmt',wb_name)
        #save_to_excel(RD.apcf_original_structure[asset_status],'cf_o_structure',wb_name)
        logger.info('Collecting CF for asset_status {0} is done.'.format(asset_status))  

INFO:__main__:Collecting CF for asset_status 正常贷款
INFO:AssetsCashFlow:cash_flow_collection...
INFO:abs_util.util_general:Saving cf_o
INFO:__main__:Collecting CF for asset_status 正常贷款 is done.
INFO:__main__:No Assets to calc for 拖欠1-30天贷款
INFO:__main__:No Assets to calc for 拖欠31-60天贷款
INFO:__main__:No Assets to calc for 拖欠61-90天贷款
INFO:__main__:No Assets to calc for 拖欠90天以上贷款


In [11]:
#'M0_2_M1':0.022,
#'M1_2_M0M2':0.5,
scenarios = {}
scenarios['best'] = {'M0_2_ERM0':0.99,'M2_2_M0M3':0.6,'M3_2_M0D':0.75,'D_2_RL':0.85,'scenario_weight':0.7}
#scenarios['worst'] = {'M0_2_ERM0':0.99,'M0_2_M1':0.05,'M1_2_M0M2':0.5,'M2_2_M0M3':0.7,'M3_2_M0D':0.7,'D_2_RL':0.99,'scenario_weight':0.3} 

In [12]:
def adjust_n_plot(best_M0_2_M1,best_M1_2_M0M2):
    
    scenarios['best']['M0_2_M1'] = best_M0_2_M1
    scenarios['best']['M1_2_M0M2'] = best_M1_2_M0M2
    
    RD.init_oAP_scenarios(scenarios)
    
    for asset_status in all_asset_status:
        if len(RD.asset_pool[(RD.asset_pool['贷款状态'] == asset_status)]) == 0:
            logger.info('No Assets to calc for {0}'.format(asset_status))
            continue
        else:
            for scenario_id in scenarios.keys():
                logger.info('Adjust oAPCF with simulation for scenario {0}...'.format(scenario_id))
                logger.info('There are total {0} simulations to go'.format(simulation_times))
                for _sim in range(simulation_times):#
                    logger.info("Calc 'Scenario - Simulation' : {0} - {1}...".format(scenario_id,_sim+1))

                    ##################
                    RD.adjust_oAPCF(scenario_id,asset_status,asset_status_calcDate_BackMonth[asset_status]['calcDate'])                
                    save_to_excel(RD.APCF_adjusted_save[asset_status][scenario_id],'cf_O_adjusted_'+scenario_id,wb_name)
                    logger.info("Calc 'Scenario - Simulation' : {0} - {1} is done.".format(scenario_id,_sim+1))

                    ######### create RD.APCF_adjusted_save[asset_status][scenario_id][_sim] to descope this function
                    RD.update_oAP_Acc(scenario_id,asset_status)
                    logger.info("Update oAP_Acc 'Scenario - Simulation' : {0} - {1} is done.".format(scenario_id,_sim+1))



    for scenario_id in scenarios.keys():

        RD.oAP_Acc_DeSimulation(scenario_id,simulation_times)
        save_to_excel(RD.df_AP_PAcc_actual_O_DeSimu[scenario_id],'De-Sim_'+scenario_id,wb_name)

    plt.figure(figsize=(8,5))
    plt.plot(RD.df_AP_PAcc_actual_O_DeSimu['best']['date_recycle'],RD.df_AP_PAcc_actual_O_DeSimu['best']['principal_loss_allTerm']/10000,c='k',lw=2)
    plt.scatter(RD.df_AP_PAcc_actual_O_DeSimu['best']['date_recycle'],RD.df_AP_PAcc_actual_O_DeSimu['best']['principal_loss_allTerm']/10000,edgecolors='k',c='yellow',s=60)
    plt.grid(True)
    plt.show()
    return (RD.df_AP_PAcc_actual_O_DeSimu['best']['date_recycle'],RD.df_AP_PAcc_actual_O_DeSimu['best']['principal_loss_allTerm']/10000)

In [13]:
p=interactive(adjust_n_plot,best_M0_2_M1 = (0.01,0.05,0.01),best_M1_2_M0M2 = (0.5,0.6,0.02))
display(p)

interactive(children=(FloatSlider(value=0.02, description='best_M0_2_M1', max=0.05, min=0.01, step=0.01), Floa…

In [21]:
for scenario_id in scenarios.keys():
    logger.info('CDR_calc_O...for {0}...'.format(scenario_id))
    
    ###############
    RD.CDR_calc_O(scenario_id)
    
save_to_excel(pd.DataFrame.from_dict(RD.CDR_O),'RnR&CDR',wb_name)

INFO:__main__:CDR_calc_O...for best...
INFO:Deal:CDR for best is: 2.3157% 
INFO:abs_util.util_general:Saving RnR&CDR


In [22]:
RD.get_rAPCF_structure()

RD.init_rAP_scenarios(scenarios)

RD.init_rAP_Acc()

for scenario_id in scenarios.keys():
    logger.info('forcast_Revolving_APCF for scenario_id {0}...'.format(scenario_id))  
    if RD.RevolvingDeal is not True:
        pass
    else:
            for which_revolving_pool in range(1,len(RD.date_revolving_pools_cut) + 1):
                
                ##################
                #logger.info('forcast_Revolving_APCF for which_revolving_pool {0}...'.format(which_revolving_pool))
                purchase_amount = RD.prepare_PurchaseAmount(which_revolving_pool,scenario_id)
                apcf_revolving_structure = deepcopy(RD.apcf_revolving_structure)

                ################
                RD.prepare_revolving_pool(scenario_id,which_revolving_pool,purchase_amount,apcf_revolving_structure)

                ##################
                APCFa = APCF_adjuster(RD.scenarios,scenario_id,RD.apcf_revolving[scenario_id][which_revolving_pool],
                                      RD.df_ppmt[scenario_id][which_revolving_pool],RD.df_ipmt[scenario_id][which_revolving_pool],
                                      RD.dates_recycle_list_revolving[scenario_id][which_revolving_pool],date_revolving_pools_cut[which_revolving_pool-1],asset_status_for_revolving)
                RD.apcf_revolving_adjusted[scenario_id][which_revolving_pool],RD.APCF_R_adjusted_save[scenario_id][which_revolving_pool] = APCFa.adjust_APCF('R')
                #save_to_excel(self.apcf_revolving_adjusted[scenario_id][which_revolving_pool],'rAPCFa_' + scenario_id + str(which_revolving_pool),wb_name)

                ###################
                RD.update_rAP_acc(scenario_id,which_revolving_pool)

INFO:RevolvingDeal:get_rAPCF_structure...
INFO:RevolvingDeal:init_rAP_Acc...
INFO:RevolvingDeal:init_rAP_Acc is done.
INFO:__main__:forcast_Revolving_APCF for scenario_id best...
INFO:RevolvingDeal:purchase_amount for scenario_id best and Revolving pool 1 is :968,273,989.24
INFO:APCF_adjuster:adjust_APCF for R...
INFO:RevolvingDeal:update_rAP_acc for revolving pool 1...
INFO:RevolvingDeal:purchase_amount for scenario_id best and Revolving pool 2 is :374,351,391.25
INFO:APCF_adjuster:adjust_APCF for R...
INFO:RevolvingDeal:update_rAP_acc for revolving pool 2...
INFO:RevolvingDeal:purchase_amount for scenario_id best and Revolving pool 3 is :429,861,930.02
INFO:APCF_adjuster:adjust_APCF for R...
INFO:RevolvingDeal:update_rAP_acc for revolving pool 3...
INFO:RevolvingDeal:purchase_amount for scenario_id best and Revolving pool 4 is :443,460,749.64
INFO:APCF_adjuster:adjust_APCF for R...
INFO:RevolvingDeal:update_rAP_acc for revolving pool 4...
INFO:RevolvingDeal:purchase_amount for scenar

In [23]:
for scenario_id in scenarios.keys():
    logger.info('forcast_Revolving_APCF for scenario_id {0}...'.format(scenario_id)) 
    RD.adjusted_all_rAP(scenario_id)
    
    save_to_excel(pd.DataFrame.from_dict(RD.apcf_revolving_adjusted_all[scenario_id]),'Adjusted_all_rAP',wb_name)


for scenario_id in scenarios.keys():
    logger.info('forcast_Revolving_APCF for scenario_id {0}...'.format(scenario_id))  
    
    ##################
    RD.CDR_calc_R(scenario_id)
    
    save_to_excel(pd.DataFrame.from_dict(RD.CDR_R[scenario_id]),'RnR&CDR',wb_name)


for scenario_id in scenarios.keys():
    RD.CDR_calc_All(scenario_id)

save_to_excel(pd.DataFrame.from_dict(RD.CDR_all),'RnR&CDR',wb_name)

INFO:__main__:forcast_Revolving_APCF for scenario_id best...
INFO:abs_util.util_general:Saving Adjusted_all_rAP
INFO:__main__:forcast_Revolving_APCF for scenario_id best...
INFO:RevolvingDeal:CDR for best is: 2.9041% for Revolving Pool 1 
INFO:RevolvingDeal:CDR for best is: 4.8166% for Revolving Pool 2 
INFO:RevolvingDeal:CDR for best is: 4.0018% for Revolving Pool 3 
INFO:RevolvingDeal:CDR for best is: 3.3100% for Revolving Pool 4 
INFO:RevolvingDeal:CDR for best is: 3.8383% for Revolving Pool 5 
INFO:RevolvingDeal:CDR for best is: 4.9598% for Revolving Pool 6 
INFO:abs_util.util_general:Saving RnR&CDR
INFO:RevolvingDeal:CDR_all for best is: 3.1438% 
INFO:abs_util.util_general:Saving RnR&CDR


In [24]:
#############
RD.get_AP_pay_buy_allocation()    

INFO:Deal:scenario_id is best


In [25]:
#fees['trustee']['rate'] = 0.00042

In [26]:
RD.init_Liability_Acc(fees)

INFO:Deal:init_Liability_Acc...


In [27]:
#RD.run_WaterFall()    # RD.waterfall[scenario_id] is available
for scenario_id in scenarios.keys():
    logger.info('scenario_id is {0}'.format(scenario_id))
    RD.waterfall[scenario_id],RD.reserveAccount_used[scenario_id] = run_Accounts(RD.AP_PAcc_original[scenario_id],RD.AP_PAcc_actual[scenario_id],
                                               RD.AP_PAcc_pay[scenario_id],RD.AP_PAcc_buy[scenario_id],
                                               RD.AP_IAcc_original[scenario_id],RD.AP_IAcc_actual[scenario_id],
                                               RD.AP_IAcc_pay[scenario_id],RD.AP_IAcc_buy[scenario_id],
                                               RD.AP_PAcc_outstanding[scenario_id],RD.AP_PAcc_reserve[scenario_id],#self.AP_PAcc_loss_allTerm[scenario_id],
                                               scenario_id,RD.RevolvingDeal,RD.Liability_Acc,RD.RevolvingPool_PurchaseAmount
                                               )

    RD.wf_BasicInfo[scenario_id] = deepcopy(BasicInfo_calculator(RD.waterfall[scenario_id],dt_param,Bonds))
    RD.wf_CoverRatio[scenario_id] = deepcopy(CR_calculator(RD.waterfall[scenario_id],RD.AP_PAcc_pay[scenario_id],RD.AP_IAcc_pay[scenario_id]))
    RD.wf_NPVs[scenario_id] = deepcopy(NPV_calculator(RD.waterfall[scenario_id],RD.AP_PAcc_pay[scenario_id],RD.AP_IAcc_pay[scenario_id]))
    RD.reserveAccount_used[scenario_id] = pd.DataFrame.from_dict(RD.reserveAccount_used[scenario_id])

for scenario_id in scenarios.keys():
    logger.info('Saving results for scenario {0} '.format(scenario_id))
    save_to_excel(RD.waterfall[scenario_id],scenario_id,wb_name)
    save_to_excel(RD.wf_BasicInfo[scenario_id],scenario_id,wb_name)
    save_to_excel(RD.wf_CoverRatio[scenario_id],scenario_id,wb_name)
    save_to_excel(RD.wf_NPVs[scenario_id],scenario_id,wb_name)
    save_to_excel(RD.reserveAccount_used[scenario_id],scenario_id,wb_name)
    save_to_excel(RD.AP_pay_buy_allocation[scenario_id],scenario_id,wb_name)

#################
RnR = RD.cal_RnR()

logger.info('RnR is: %s' % RnR)
save_to_excel(pd.DataFrame({'RnR':[RnR]}),'RnR&CDR',wb_name)

INFO:__main__:scenario_id is best
INFO:abs_util.util_waterfall:run_Accounts...
INFO:__main__:Saving results for scenario best 
INFO:abs_util.util_general:Saving best
INFO:abs_util.util_general:Saving best
INFO:abs_util.util_general:Saving best
INFO:abs_util.util_general:Saving best
INFO:abs_util.util_general:Saving best
INFO:abs_util.util_general:Saving best
INFO:__main__:RnR is: 0.26448803049968467
INFO:abs_util.util_general:Saving RnR&CDR
